In [41]:
import pandas as pd

In [42]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [43]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [44]:
## Drop Nan values
df = df.dropna()

In [45]:
## Get independent & dependent features
X = df.drop('label', axis=1)
y = df['label']

In [46]:
import tensorflow as tf

In [47]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [48]:
## Vocabulary size
voc_size = 5000

In [49]:
## One hot Representation
messages = X.copy()

In [50]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [51]:
import nltk
import re
from nltk.corpus import stopwords

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    # Remove non-alphabetic characters
    review = re.sub('[^a-zA-Z]', " ", messages['title'].iloc[i])
    # Convert to lowercase
    review = review.lower()
    # Tokenize
    review = review.split()
    # Remove stopwords and apply stemming
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    # Rejoin tokens
    review = ' '.join(review)
    corpus.append(review)

In [54]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [55]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[2568, 2368, 1438, 4548, 3468, 4662, 2848, 3354, 1218, 2439],
 [4308, 1335, 67, 4005, 3670, 685, 2599],
 [1067, 353, 1462, 1034],
 [461, 2560, 1041, 4886, 3073, 2616],
 [3931, 3670, 400, 136, 369, 2872, 3670, 4556, 2557, 2289],
 [3465,
  1570,
  2288,
  198,
  1289,
  1753,
  3113,
  1693,
  1987,
  1546,
  4371,
  85,
  4831,
  3731,
  2599],
 [2146, 4544, 2495, 805, 3501, 305, 2574, 3119, 2016, 4970, 2845],
 [4856, 817, 4285, 2122, 2226, 2192, 1753, 2717, 2016, 4970, 2845],
 [2957, 3391, 2302, 4214, 18, 3100, 3244, 1825, 1753, 2881],
 [3864, 76, 2503, 4981, 2628, 2220, 3850, 1348],
 [3553, 4241, 4560, 924, 1083, 502, 831, 4712, 205, 3320, 2051],
 [4886, 4211, 3468, 3100, 1753, 2226],
 [3126, 4582, 2248, 4824, 3284, 2939, 1275, 1607, 3578],
 [4941, 3183, 687, 4243, 4385, 2564, 3818, 2016, 4970, 2845],
 [4929, 3343, 1894, 1968, 4961, 2016, 4970, 2845],
 [1886, 2027, 1855, 553, 4621, 969, 2397, 4983, 3138, 3595],
 [3107, 91, 1335],
 [1759, 2492, 1780, 4051, 1753, 3334, 1128, 2599],
 [2

In [56]:
onehot_repr[1]

[4308, 1335, 67, 4005, 3670, 685, 2599]

In [57]:
## Word2Vec to Embedding layer
sent_length = 30
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3354 1218 2439]
 [   0    0    0 ... 3670  685 2599]
 [   0    0    0 ...  353 1462 1034]
 ...
 [   0    0    0 ... 2016 4970 2845]
 [   0    0    0 ... 4644  679 1122]
 [   0    0    0 ... 1915 1460 1525]]


In [58]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 2568, 2368,
       1438, 4548, 3468, 4662, 2848, 3354, 1218, 2439])

In [59]:
##Creating the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [60]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [63]:
##fittinf the training
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - accuracy: 0.7700 - loss: 0.4254 - val_accuracy: 0.9099 - val_loss: 0.2041
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9404 - loss: 0.1453 - val_accuracy: 0.9138 - val_loss: 0.2014
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.9667 - loss: 0.0966 - val_accuracy: 0.9157 - val_loss: 0.2217
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.9775 - loss: 0.0663 - val_accuracy: 0.9105 - val_loss: 0.2643
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.9872 - loss: 0.0439 - val_accuracy: 0.9157 - val_loss: 0.2857
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9936 - loss: 0.0250 - val_accuracy: 0.9150 - val_loss: 0.3894
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.9947 - loss: 0.0180 - val_accuracy: 0.9105 - val_loss: 0.4221
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.9973 - loss: 0.0110 - val_accu

In [64]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred>0.5, 1, 0)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3111,  308],
       [ 231, 2385]], dtype=int64)

In [66]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9106876553438277

In [67]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

